In [142]:

import statsmodels.formula.api as smf
import pandas as pd
import sys
sys.path.append('../src/')

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# Load Data

In [165]:
df0 = {
    '1':pd.read_csv('../data/exp_1.csv'),
    '2':pd.read_csv('../data/exp_2.csv'),
    '3':pd.read_csv('../data/exp_3.csv'),
    '4':pd.read_csv('../data/exp_4.csv')
}

# Prepare DataFrame

The `prep_df()` function performs all required integrations

In [160]:
def prep_df(df_exp, orders, reactants, catalysts):
    for key in df_exp.keys():
        kobs_cols = []
        for col in df_exp[key].columns:
            if col in reactants + ['t']:
                df_exp[key][f'{col}-1'] = df_exp[key][col].shift(1)
            if col in reactants:
                kobs_cols.append(f'[{col}]^{orders[col]}')
                df_exp[key][f'[{col}]^{orders[col]}'] = 0.5*((df_exp[key][f'{col}'] + df_exp[key][f'{col}-1']).pow(orders[col]))
                df_exp[key][f'[{col}]^{orders[col]}∆t'] = 0.5*((df_exp[key][f'{col}'] + df_exp[key][f'{col}-1']).pow(orders[col]))*(df_exp[key]['t'] - df_exp[key]['t-1'])
                df_exp[key][f'∑[{col}]^{orders[col]}∆t'] = df_exp[key][f'[{col}]^{orders[col]}∆t'].cumsum()
            for cat_name in catalysts:
                kobs_cols.append(f'[{cat_name}]^{orders[cat_name]}')
                df_exp[key][f'[{cat_name}]^{orders[cat_name]}'] = df_exp[key][cat_name].pow(orders[cat_name])
                df_exp[key][f't[{cat_name}]^{orders[cat_name]}'] = df_exp[key]['t']*df_exp[key][cat_name].pow(orders[cat_name])
        
        kobs_cols.append('∆t')
        df_exp[key]['∆t'] = df_exp[key]['t'] - df_exp[key]['t-1']
        
        df_exp[key]['∑'+str.join('',[f'[{col}]^{orders[col]}' for col in reactants + catalysts])+'∆t'] = df_exp[key][list(set(kobs_cols))].product(axis=1, skipna=False).cumsum()
     
    return df_exp



In [166]:
prep_df(df0, orders, reactants, catalysts)['1']

,t,A,B,P,cat,t-1,[cat]^1,t[cat]^1,A-1,[A]^1,[A]^1∆t,∑[A]^1∆t,B-1,[B]^1,[B]^1∆t,∑[B]^1∆t,∆t,∑[A]^1[B]^1[cat]^1∆t
0,0.00,0.998,0.600,0.001,0.01,NaN,0.01,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.89,0.850,0.449,0.150,0.01,0.00,0.01,0.0089,0.998,0.9240,0.822360,0.822360,0.600,0.5245,0.466805,0.466805,0.89,0.004313
2,2.03,0.726,0.326,0.273,0.01,0.89,0.01,0.0203,0.850,0.7880,0.898320,1.720680,0.449,0.3875,0.441750,0.908555,1.14,0.007794
3,3.89,0.609,0.209,0.388,0.01,2.03,0.01,0.0389,0.726,0.6675,1.241550,2.962230,0.326,0.2675,0.497550,1.406105,1.86,0.011115
4,6.61,0.522,0.123,0.477,0.01,3.89,0.01,0.0661,0.609,0.5655,1.538160,4.500390,0.209,0.1660,0.451520,1.857625,2.72,0.013669
5,10.00,0.468,0.067,0.534,0.01,6.61,0.01,0.1000,0.522,0.4950,1.678050,6.178440,0.123,0.0950,0.322050,2.179675,3.39,0.015263
6,14.03,0.435,0.036,0.564,0.01,10.00,0.01,0.1403,0.468,0.4515,1.819545,7.997985,0.067,0.0515,0.207545,2.387220,4.03,0.016200
7,18.36,0.417,0.018,0.582,0.01,14.03,0.01,0.1836,0.435,0.4260,1.844580,9.842565,0.036,0.0270,0.116910,2.504130,4.33,0.016698
8,22.97,0.409,0.009,0.592,0.01,18.36,0.01,0.2297,0.417,0.4130,1.903930,11.746495,0.018,0.0135,0.062235,2.566365,4.61,0.016955
9,27.78,0.403,0.004,0.595,0.01,22.97,0.01,0.2778,0.409,0.4060,1.952860,13.699355,0.009,0.0065,0.031265,2.597630,4.81,0.017082


# Figure 2 a)

In [181]:
reactants = ['A', 'B']
catalysts = ['cat']
orders = {'A':1, 'B':1, 'cat':1}

df_exp = prep_df(df0, orders, reactants, catalysts)

plot = figure(width=600,height=250)
plot.circle(x=f'∑[A]^{orders["A"]}∆t', y='P', color='blue', source=ColumnDataSource(df_exp['1']))
plot.triangle(x=f'∑[A]^{orders["A"]}∆t', y='P', color='orange', source=ColumnDataSource(df_exp['2']))
plot.add_tools(HoverTool(tooltips=[('A', '@A{0.000}'), ('B', '@B{0.000}'), ('P', '@P{0.000}')]))
show(plot)

# Figure 2 b)

In [182]:
reactants = ['A', 'B']
catalysts = ['cat']
orders = {'A':1, 'B':1, 'cat':1}

df_exp = prep_df(df0, orders, reactants, catalysts)

plot = figure(width=600,height=250)
plot.circle(x=f'∑[B]^{orders["B"]}∆t', y='P', color='blue', source=ColumnDataSource(df_exp['1']))
plot.square(x=f'∑[B]^{orders["B"]}∆t', y='P', color='green', source=ColumnDataSource(df_exp['3']))
plot.add_tools(HoverTool(tooltips=[('A', '@A{0.000}'), ('B', '@B{0.000}'), ('P', '@P{0.000}')]))
show(plot)

# Figure 2 c)

In [183]:
reactants = ['A', 'B']
catalysts = ['cat']
orders = {'A':1, 'B':1, 'cat':1}

df_exp = prep_df(df0, orders, reactants, catalysts)

plot = figure(width=600,height=250)
plot.circle(x='t[cat]^1', y='P', color='blue', source=ColumnDataSource(df_exp['1']))
plot.diamond(x='t[cat]^1', y='P', color='red', source=ColumnDataSource(df_exp['4']))
plot.add_tools(HoverTool(tooltips=[('A', '@A{0.000}'), ('B', '@B{0.000}'), ('P', '@P{0.000}')]))
show(plot)

# Figure 2 d)

In [184]:
reactants = ['A', 'B']
catalysts = ['cat']
orders = {'A':1, 'B':1, 'cat':1}

df_exp = prep_df(df0, orders, reactants, catalysts)

plot = figure(width=600,height=250)
plot.circle(x=df_exp['1'].columns[-1], y='P', color='blue', source=ColumnDataSource(df_exp['1']))
plot.triangle(x=df_exp['2'].columns[-1], y='P', color='orange', source=ColumnDataSource(df_exp['2']))
plot.square(x=df_exp['3'].columns[-1], y='P', color='green', source=ColumnDataSource(df_exp['3']))
plot.diamond(x=df_exp['4'].columns[-1], y='P', color='red', source=ColumnDataSource(df_exp['4']))
plot.add_tools(HoverTool(tooltips=[('A', '@A{0.000}'), ('B', '@B{0.000}'), ('P', '@P{0.000}')]))
show(plot)

# Determine K_obs using OLS

In [173]:
# Prepare df for regression
df_reg = pd.concat([df_exp[f'{i}'][['P', df_exp['1'].columns[-1]]] for i in [1,2,3,4]]).dropna()

# Rename columns
df_reg.columns = ['P', 'x'] 

In [172]:
smf.ols('P ~ x-1', data=df_reg).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      P   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.327e+05
Date:                Tue, 12 Feb 2019   Prob (F-statistic):           3.46e-78
Time:                        19:08:41   Log-Likelihood:                 156.22
No. Observations:                  36   AIC:                            -310.4
Df Residuals:                      35   BIC:                            -308.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x             34.8277      0.038    912.498      0.000      34.750      34.905
==============================================================================
Omnibus:                       22.711   Durbin-Watson:                   1.270
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.440
Skew:                          -1.573   Prob(JB):                     1.65e-09
Kurtosis:                       7.130   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""